In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf

In [5]:
num_frame = 20
directory = "txt_"

X = []
y = []
file_list = os.listdir(directory)
for file in file_list:
    df = pd.read_csv(os.path.join(directory,file),index_col=0)
    dataset = df.iloc[:]
    label = int(file.split("_")[0])
    n_sample = len(dataset)
    for i in range(num_frame,n_sample):
        X.append(dataset[i-num_frame:i])
        y.append(label)

X = np.array(X)
y = np.array(y)


In [6]:
from sklearn.utils import shuffle
X, y = shuffle(X,y)

def normalize(X):
    result = []
    for rec in X:
        result.append((rec - np.mean(rec))/np.std(rec))
    return np.array(result)

# X, X_test = normalize(X), normalize(X_test)

In [7]:
class ECA(tf.keras.layers.Layer):
    def __init__(self, kernel_size=5, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.kernel_size = kernel_size
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)

    def call(self, inputs, mask=None):
        nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
        nn = tf.expand_dims(nn, -1)
        nn = self.conv(nn)
        nn = tf.squeeze(nn, -1)
        nn = tf.nn.sigmoid(nn)
        nn = nn[:,None,:]
        return inputs * nn

class LateDropout(tf.keras.layers.Layer):
    def __init__(self, rate, noise_shape=None, start_step=0, **kwargs):
        super().__init__(**kwargs)
        self.supports_masking = True
        self.rate = rate
        self.start_step = start_step
        self.dropout = tf.keras.layers.Dropout(rate, noise_shape=noise_shape)
      
    def build(self, input_shape):
        super().build(input_shape)
        agg = tf.VariableAggregation.ONLY_FIRST_REPLICA
        self._train_counter = tf.Variable(0, dtype="int64", aggregation=agg, trainable=False)

    def call(self, inputs, training=False):
        x = tf.cond(self._train_counter < self.start_step, lambda:inputs, lambda:self.dropout(inputs, training=training))
        if training:
            self._train_counter.assign_add(1)
        return x


In [8]:
class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self, 
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer='glorot_uniform',
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True
        
    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

def Conv1DBlock(channel_size,
          kernel_size,
          dilation_rate=1,
          drop_rate=0.0,
          expand_ratio=2,
          se_ratio=0.25,
          activation='swish',
          name=None):
    '''
    efficient conv1d block, @hoyso48
    '''
    if name is None:
        name = str(tf.keras.backend.get_uid("mbblock"))
    # Expansion phase
    def apply(inputs):
        channels_in = tf.keras.backend.int_shape(inputs)[-1]
        channels_expand = channels_in * expand_ratio

        skip = inputs

        x = tf.keras.layers.Dense(
            channels_expand,
            use_bias=True,
            activation=activation,
            name=name + '_expand_conv')(inputs)

        # Depthwise Convolution
        x = CausalDWConv1D(kernel_size,
            dilation_rate=dilation_rate,
            use_bias=False,
            name=name + '_dwconv')(x)

        x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

        x  = ECA()(x)

        x = tf.keras.layers.Dense(
            channel_size,
            use_bias=True,
            name=name + '_project_conv')(x)

        if drop_rate > 0:
            x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

        if (channels_in == channel_size):
            x = tf.keras.layers.add([x, skip], name=name + '_add')
        return x

    return apply

In [35]:
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, dim=256, num_heads=4, dropout=0, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.scale = self.dim ** -0.5
        self.num_heads = num_heads
        self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
        self.drop1 = tf.keras.layers.Dropout(dropout)
        self.proj = tf.keras.layers.Dense(dim, use_bias=False)
        self.supports_masking = True

    def call(self, inputs, mask=None):
        qkv = self.qkv(inputs)
        qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
        q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

        attn = tf.matmul(q, k, transpose_b=True) * self.scale

        if mask is not None:
            mask = mask[:, None, None, :]

        attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
        attn = self.drop1(attn)

        x = attn @ v
        x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
        x = self.proj(x)
        return x


def TransformerBlock(dim=256, num_heads=4, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
    def apply(inputs):
        x = inputs
        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([inputs, x])
        attn_out = x

        x = tf.keras.layers.BatchNormalization(momentum=0.95)(x)
        x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
        x = tf.keras.layers.Dense(dim, use_bias=False)(x)
        x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
        x = tf.keras.layers.Add()([attn_out, x])
        return x
    return apply

In [76]:
MAX_LEN = 20 # number of frame
CHANNELS = 195 # number of keypoint value
NUM_CLASSES = 87

# ----------------------------------------- DEFINE MODEL -----------------------------
def get_model(max_len=MAX_LEN, dropout_step=0, dim=192):
    inp = tf.keras.Input((max_len,CHANNELS))
    #x = tf.keras.layers.Masking(mask_value=PAD,input_shape=(max_len,CHANNELS))(inp) #we don't need masking layer with inference
    x = inp
    ksize = 3
    # x = tf.keras.layers.Permute((2,1))(x)
    # x = tf.keras.layers.Dense(dim, use_bias=False,name='stem_conv')(x)
    # x = tf.keras.layers.BatchNormalization(momentum=0.95,name='stem_bn')(x)

    # x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    # x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.3)(x)
    x = TransformerBlock(dim,expand=2)(x)

    # x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    # x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    x = Conv1DBlock(dim,ksize,drop_rate=0.3)(x)
    x = TransformerBlock(dim,expand=2)(x)

    # if dim == 384: #for the 4x sized model
    #     x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    #     x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    #     x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    #     x = TransformerBlock(dim,expand=2)(x)

    #     x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    #     x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    #     x = Conv1DBlock(dim,ksize,drop_rate=0.2)(x)
    #     x = TransformerBlock(dim,expand=2)(x)

    x = tf.keras.layers.Dense(dim*2,activation=None,name='top_conv')(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    # x = LateDropout(0.2, start_step=dropout_step)(x)
    x = tf.keras.layers.Dense(NUM_CLASSES,name='classifier',activation="softmax")(x)
    return tf.keras.Model(inp, x)

In [77]:
model = get_model()
model.summary()

Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_15      │ (None, 20, 195)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 44_expand_conv      │ (None, 20, 390)   │     76,440 │ input_layer_15[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 44_dwconv           │ (None, 20, 390)   │      1,170 │ 44_expand_conv[0… │
│ (CausalDWConv1D)    │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 44_bn               │ (None, 20, 390)   │      1,560 │ 44_dwconv[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ eca_43 (ECA)        │ (None, 20, 390)   │          5 │ 44_bn[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 44_project_conv     │ (None, 20, 192)   │     75,072 │ eca_43[0][0]      │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 44_drop (Dropout)   │ (None, 20, 192)   │          0 │ 44_project_conv[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 192)   │        768 │ 44_drop[0][0]     │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_self_at… │ (None, 20, 192)   │    147,456 │ batch_normalizat… │
│ (MultiHeadSelfAtte… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_59          │ (None, 20, 192)   │          0 │ multi_head_self_… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_33 (Add)        │ (None, 20, 192)   │          0 │ 44_drop[0][0],    │
│                     │                   │            │ dropout_59[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 20, 192)   │        768 │ add_33[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_70 (Dense)    │ (None, 20, 384)   │     73,728 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_71 (Dense)    │ (None, 20, 192)   │     73,728 │ dense_70[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_60          │ (None, 20, 192)   │          0 │ dense_71[0][0]    │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_34 (Add)        │ (None, 20, 192)   │          0 │ add_33[0][0],     │
│                     │                   │            │ dropout_60[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 45_expand_conv      │ (None, 20, 384)   │     74,112 │ add_34[0][0]      │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ 45_dwconv           │ (None, 20, 384)   │      1,152 │ 45_expand_conv[0… │
│ (CausalDWConv1D)    │                   │            │                 

 Total params: 1,005,475 (3.84 MB)

 Trainable params: 1,002,391 (3.82 MB)

 Non-trainable params: 3,084 (12.05 KB)

In [78]:
from keras.callbacks import LearningRateScheduler, ModelCheckpoint, EarlyStopping
from keras.optimizers import AdamW

def scheduler(epoch,lr):
    return lr*0.9
lr_scheduler = LearningRateScheduler(scheduler)
checkpoint = ModelCheckpoint(
    filepath="1DCNN_Transformer.weights.h5",
    save_best_only=True,
    save_weights_only=True,
)
early_stopping = EarlyStopping(monitor='val_loss', patience=20, start_from_epoch=20)

model.compile(loss="categorical_crossentropy",
              metrics=["accuracy"],
              optimizer=AdamW(1e-4))

In [29]:
from keras.utils import to_categorical
y= to_categorical(y,87)

In [79]:
num_frame = 20
directory = "pre_split_test_files/txt_"

X_test = []
y_test = []
file_list = os.listdir(directory)
for file in file_list:
    df = pd.read_csv(os.path.join(directory,file),index_col=0)
    dataset = df.iloc[:]
    label = int(file.split("_")[0])
    n_sample = len(dataset)
    for i in range(num_frame,n_sample):
        X_test.append(dataset[i-num_frame:i])
        y_test.append(label)

X_test = np.array(X_test)
y_test = np.array(y_test)
y_test = to_categorical(y_test)

In [80]:
batch_size = 128
epochs = 50

history = model.fit(X,y,
                    validation_data=(X_test,y_test),
                    epochs=epochs,
                    batch_size=batch_size,
                    callbacks=[lr_scheduler,early_stopping,checkpoint])

Epoch 1/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 32s 115ms/step - accuracy: 0.1074 - loss: 3.9884 - val_accuracy: 0.0719 - val_loss: 3.6650 - learning_rate: 9.0000e-05
Epoch 2/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 26s 115ms/step - accuracy: 0.3159 - loss: 2.8065 - val_accuracy: 0.1445 - val_loss: 3.2161 - learning_rate: 8.1000e-05
Epoch 3/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 26s 117ms/step - accuracy: 0.4194 - loss: 2.4270 - val_accuracy: 0.1650 - val_loss: 3.0520 - learning_rate: 7.2900e-05
Epoch 4/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 24s 109ms/step - accuracy: 0.4829 - loss: 2.1958 - val_accuracy: 0.2185 - val_loss: 2.8931 - learning_rate: 6.5610e-05
Epoch 5/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 25s 111ms/step - accuracy: 0.5151 - loss: 2.0900 - val_accuracy: 0.2091 - val_loss: 3.0032 - learning_rate: 5.9049e-05
Epoch 6/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 25s 110ms/step - accuracy: 0.5489 - loss: 1.9565 - val_accuracy: 0.2207 - val_loss: 3.0164 - learning_rate: 5.3144e-05
Epoch 7/50
223/223 ━━━━━━━━━━━━━━━━━━━━ 24s 10

KeyboardInterrupt: 